In [1]:
import pandas as pd
import random
import os

/Users/edmondniu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Enter Subject Number ##

In [2]:
#Subject Number
subjN = 25

## Import Mapping Data and Clean It ##

In [3]:
mapping = pd.read_csv("Mapping/Misophonia Mapping Sounds 2.csv")
mapping = mapping[['Name', 'Number']]
mapping.columns = ['Name', 'Sound']

## Import Qualtrics Form Data ##

In [4]:
# Data provided by the user

# Creating the DataFrame
df = pd.read_csv("Raw Data/miso raw data 10.csv")
df = df.drop(4)

df['subject_number']

#5: NaNs (didnt finish)
#4: no start date (order 73 didn't work)
#25: finished (order 64 didnt work) - 4 aversive, 5 miso
#6: Finished (no errors)
#15: (2 miso, 6 aversive, sound#78 didnt play

0     Please enter your subject number:
1             {"ImportId":"QID13_TEXT"}
2                                     5
3                                     6
5                                     4
6                                    25
7                                    18
8                                    30
9                                     2
10                                    3
11                                    9
12                                   15
Name: subject_number, dtype: object

## Manipulate / Wrangle the Input Qualtrics Data ##

In [5]:
#remove all rows that are not the current subject
df.rename(columns={'subject_number': 'Subject Number'}, inplace=True)
df = df[df['Subject Number'] == str(subjN)]

if df.shape[0] == 0:
    print("Error: Subject Number: " + str(subjN) + " not found!")

#print(columns_list)

In [6]:
#get date
date = df['StartDate'].iloc[0]
date = date[:10]

#remove uneccessary columns
columns_list = df.columns.tolist()
columns_to_remove = ['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage', 'Browser Type_Browser', 'Browser Type_Version', 'Browser Type_Operating System', 'Browser Type_Resolution', 'sounds', 'txtFile', 'subject_numbers', 'foundSubject', 'Order', 'EndLoop']
df = df.drop(columns=columns_to_remove)

df

,Subject Number,1_Order,1_Rating_1,1_Memory,1_Trigger,2_Order,2_Rating_1,2_Memory,2_Trigger,3_Order,...,107_Memory,107_Trigger,108_Order,108_Rating_1,108_Memory,108_Trigger,109_Order,109_Rating_1,109_Memory,109_Trigger
6,25,50,1,Yes,NaN,32,0,No,NaN,44,...,Yes,NaN,33,3,No,NaN,16,8,No,NaN


In [7]:
# Pivoting the DataFrame to match the desired format

# Melting the dataframe to make it long
df_melted = df.melt(id_vars=["Subject Number"],
                    var_name="Sound_Trigger",
                    value_name="Value")
#df_melted

In [8]:
# Splitting the Sound_Trigger column into two separate columns: one for Sound, another for Trigger/Rating
df_melted['Sound'] = df_melted['Sound_Trigger'].apply(lambda x: x.split('_')[0])
df_melted['Type'] = df_melted['Sound_Trigger'].apply(lambda x: x.split('_')[1])

In [9]:
# Pivoting the table to get the correct format
df_pivoted = df_melted.pivot_table(index=['Subject Number', 'Sound'], columns='Type', values='Value', aggfunc='first').reset_index()
#df_pivoted

In [10]:
# Renaming columns for clarity
df_final = df_pivoted.rename(columns={"Subject Number": "Subject", "Rating": "Rating", "Trigger": "Trigger"})
df_final['Sound'] = df_final['Sound'].astype(int)  # Converting Sound to numeric for proper sorting

df_final = df_final.sort_values(by=['Sound']).reset_index(drop=True)
#fix qualtrics bug
df_final.loc[df_final['Sound'] > 56, 'Sound'] -= 3

#merge mapping + df_final
df_final = pd.merge(df_final, mapping, on="Sound")

df_final = df_final[['Subject', 'Sound', 'Name', 'Rating', 'Trigger', 'Memory', 'Order']]
df_final

,Subject,Sound,Name,Rating,Trigger,Memory,Order
0,25,1,Personalized 1,1,NaN,Yes,50
1,25,2,Personalized 2,0,NaN,No,32
2,25,3,Personalized 3,-2,No,NaN,44
3,25,4,Personalized 4,-4,No,NaN,25
4,25,5,Personalized 5,0,NaN,No,77
...,...,...,...,...,...,...,...
101,25,102,n_RobinChirping_s.wav,5,NaN,Yes,68
102,25,103,n_VaccuumCleaner_s.wav,0,NaN,Yes,42
103,25,104,n_WashingMachine_s.wav,2,NaN,Yes,61
104,25,105,n_WaterStream_s.wav,3,NaN,No,33


In [11]:
#Na Rows
nan_rows = df_final[df_final['Rating'].isna()]

nan_rows

,Subject,Sound,Name,Rating,Trigger,Memory,Order


In [12]:
#Sound order 73 for subject 4: #Sound 65, IADS_vomit_255  
df_final[df_final['Order'] == str(64)]

#Sound order 64 for subject 25: #Sound 35, Coughing and Sniffing  

,Subject,Sound,Name,Rating,Trigger,Memory,Order
34,25,35,m_CoughingAndSniffing_s.wav,0,NaN,No,64


## 1. Create sound_rating_all: Sound Ratings Table for ALL Sounds ##

#### id, date, name, rating, trigger ####

In [13]:
sound_rating_all = df_final[['Subject', 'Sound', 'Name', 'Rating', 'Trigger', 'Memory', 'Order']]
sound_rating_all['Date'] = date

sound_rating_all = sound_rating_all[['Subject', 'Date', 'Order', 'Name', 'Rating', 'Trigger', 'Memory']]

sound_rating_all

,Subject,Date,Order,Name,Rating,Trigger,Memory
0,25,2024-04-17,50,Personalized 1,1,NaN,Yes
1,25,2024-04-17,32,Personalized 2,0,NaN,No
2,25,2024-04-17,44,Personalized 3,-2,No,NaN
3,25,2024-04-17,25,Personalized 4,-4,No,NaN
4,25,2024-04-17,77,Personalized 5,0,NaN,No
...,...,...,...,...,...,...,...
101,25,2024-04-17,68,n_RobinChirping_s.wav,5,NaN,Yes
102,25,2024-04-17,42,n_VaccuumCleaner_s.wav,0,NaN,Yes
103,25,2024-04-17,61,n_WashingMachine_s.wav,2,NaN,Yes
104,25,2024-04-17,33,n_WaterStream_s.wav,3,NaN,No


## 2. Create df_miso_aversive: Miso/Aversive Sound Ratings Table ##

In [14]:
#get rid of all NaN values in Trigger column (get rid of positive sounds)
df_miso_aversive = df_final.dropna(subset=['Trigger'])

#Drop Memory Column (not needed)
df_miso_aversive = df_miso_aversive.drop(columns=['Memory'])

#convert str --> int
df_miso_aversive['Rating'] = df_miso_aversive['Rating'].astype(int)

#abs value of ratings
df_miso_aversive['Rating'] = df_miso_aversive['Rating'].abs()

## 3. Create df_miso: Miso Sounds Ratings Table ##

In [15]:
df_miso = df_miso_aversive[df_miso_aversive['Trigger'] == 'Yes'].reset_index(drop=True)
#df_miso

## 4. Create df_aversive: Aversive Sounds Ratings Table ##

In [16]:
df_aversive = df_miso_aversive[df_miso_aversive['Trigger'] == 'No'].reset_index(drop=True)
#df_aversive

## 5. Create df_mri_ratings: 20 mri sounds ratings table ##

In [17]:
df_miso_10 = df_miso.sort_values(by=['Rating'], ascending = False).head(10).reset_index(drop=True)
df_aversive_10 = df_aversive.sort_values(by=['Rating'], ascending = False).head(10).reset_index(drop=True)

#### Set Warning_MRI (Make sure df_miso_10 and df_aversive_10 HAVE 10 sounds each before merging into df_mri_ratings) ####

##### Set Warning MRI Miso for df_Miso (less than 10 sounds) #####

In [18]:
warning_MRI_miso_less10 = False
warning_MRI_numRows__miso = df_miso_10.shape[0] #num miso sounds provided
#warning_MRI_numRows__miso = 9 #testing

warning_MRI_miso_sounds_added = "None" #MISO Sounds ADDED
warning_MRI_miso_sounds_repeated = "None" #MISO SoundsREPEATED
warning_MRI_num_miso_sounds_ADD_REPEAT = 0 #NUM MISO ADDED/REPEATED

df_miso_10

,Subject,Sound,Name,Rating,Trigger,Order
0,25,75,m_SlurpingEatingCombined_s.wav,7,Yes,80
1,25,65,m_IADS_Vomit_255_s.wav,6,Yes,34
2,25,49,m_EatingSmallCrunches_s.wav,4,Yes,11
3,25,59,m_HotTeaSlurping_s.wav,4,Yes,21
4,25,76,m_SlurpingNoodles_s.wav,4,Yes,109


In [19]:
#How do these warnings work:
#If there are less than 6 misophonic sounds, pull from all m_ sounds.
#Take all aversive sounds from m_ sounds and sort them from most negative to least negative (most negative --> least negative)
#Take all neut/pos sounds from m_ sounds and sort them from least positive to most positive (most negative --> least negative)
#Concat aversive --> neut/pos sounds together and take only the most negative x sounds needed to hit 10 sounds
#Coerce all sounds to be miso and change all neg values to abs values

#if there are 6 or more misophonic sounds, select randomly without repeating and hit 10 miso sounds (no more than 1 repeat per sound)

if warning_MRI_numRows__miso <= 5:
    warning_MRI_miso_less10 = True
    warning_MRI_numRepeats_miso = 10 - warning_MRI_numRows__miso #num miso sounds needed

    nontriggerMiso_sounds_Mprefix = df_final[(df_final['Sound'] >= 26) & (df_final['Sound'] <= 95)].drop('Memory', axis=1)

    aversiveRatings = nontriggerMiso_sounds_Mprefix[nontriggerMiso_sounds_Mprefix['Trigger'] == 'No']
    aversiveRatings = aversiveRatings.sort_values(by=['Rating'], ascending=True) #.head(warning_MRI_numRepeats_miso)

    posNeutRatings = nontriggerMiso_sounds_Mprefix[nontriggerMiso_sounds_Mprefix['Trigger'].isna()]

    posNeutRatings['Rating'] = posNeutRatings['Rating'].astype(int)
    posNeutRatings = posNeutRatings.sort_values(by=['Rating'], ascending=True) #.head(warning_MRI_numRepeats_miso)
    #print(posNeutRatings)
    
    aversive_posNeut_Ratings = pd.concat([aversiveRatings, posNeutRatings])
    #print(aversive_posNeut_Ratings)
    aversive_posNeut_Ratings = aversive_posNeut_Ratings.head(warning_MRI_numRepeats_miso)

    #Coerce
    aversive_posNeut_Ratings['Trigger'] = 'Yes'
    aversive_posNeut_Ratings['Rating'] = aversive_posNeut_Ratings['Rating'].astype(int)
    aversive_posNeut_Ratings['Rating'] = aversive_posNeut_Ratings['Rating'].abs()

    warning_MRI_miso_sounds_added = set(aversive_posNeut_Ratings['Name']) #which miso sounds added
    warning_MRI_num_miso_sounds_ADD_REPEAT = len(aversive_posNeut_Ratings) #num miso sound added 
    
    df_miso_10 = pd.concat([df_miso_10, aversive_posNeut_Ratings], ignore_index=True)
    
    #nontriggerMiso_sounds_Mprefix['Rating'] = nontriggerMiso_sounds_Mprefix['Rating'].astype(int)
    #nontriggerMiso_sounds_Mprefix['Rating'] = nontriggerMiso_sounds_Mprefix['Rating'].abs()
    
    #nontriggerMiso_sounds_Mprefix = pd.concat([nontriggerMiso_sounds_Mprefix, df_miso_10, df_miso_10]).drop_duplicates(keep=False)
    
    #ADD_miso_sounds_Mprefix_needed = nontriggerMiso_sounds_Mprefix.sort_values(by=['Rating'], ascending=True) #.head(warning_MRI_numRepeats_miso)
    #print(ADD_miso_sounds_Mprefix_needed) #testing 1
    #ADD_miso_sounds_Mprefix_needed['Trigger'] = 'Yes'
    #ADD_miso_sounds_Mprefix_needed['Rating'] = 0
    
    #warning_MRI_miso_sounds_added = set(ADD_miso_sounds_Mprefix_needed['Name']) #which miso sounds added
    #warning_MRI_num_miso_sounds_ADD_REPEAT = len(warning_MRI_miso_sounds_added) #num miso sound added 
    
    #df_miso_10 = pd.concat([df_miso_10, ADD_miso_sounds_Mprefix_needed], ignore_index=True)


if (warning_MRI_numRows__miso > 5) and (warning_MRI_numRows__miso < 10):
    warning_MRI_miso_less10 = True
    warning_MRI_numRepeats_miso = 10 - warning_MRI_numRows__miso #num miso sounds needed

    DUPLICATES_5to10_MRI = df_miso_10.sample(n=warning_MRI_numRepeats_miso)
    
    warning_MRI_miso_sounds_repeated = set(DUPLICATES_5to10_MRI['Name'])  #which miso sounds repeated
    warning_MRI_num_miso_sounds_ADD_REPEAT = len(warning_MRI_miso_sounds_repeated) #num miso sound added 
    
    df_miso_10 = pd.concat([df_miso_10, DUPLICATES_5to10_MRI], ignore_index=True)

df_miso_10

/var/folders/qq/qm2zshms6rl57mb9s95rh3xc0000gn/T/ipykernel_4141/4126205974.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posNeutRatings['Rating'] = posNeutRatings['Rating'].astype(int)


,Subject,Sound,Name,Rating,Trigger,Order
0,25,75,m_SlurpingEatingCombined_s.wav,7,Yes,80
1,25,65,m_IADS_Vomit_255_s.wav,6,Yes,34
2,25,49,m_EatingSmallCrunches_s.wav,4,Yes,11
3,25,59,m_HotTeaSlurping_s.wav,4,Yes,21
4,25,76,m_SlurpingNoodles_s.wav,4,Yes,109
5,25,73,m_SippingCoffeeCombined_s.wav,2,Yes,58
6,25,26,m_ALotOfYawning_s.wav,0,Yes,67
7,25,56,m_HeavyBreathing2_s.wav,0,Yes,81
8,25,57,m_HeavyBreathing_s.wav,0,Yes,95
9,25,60,m_iads_Chewing_724_s.wav,0,Yes,66


##### Set Warning MRI Aversive for df_aversive (less than 10 sounds) #####

In [20]:
warning_MRI_aver_less10 = False
warning_MRI_numRows__aversive = df_aversive_10.shape[0]
#warning_MRI_numRows__aversive = 0 #testing

warning_MRI_aver_sounds_added = "None"
warning_MRI_aver_sounds_repeated = "None"
warning_MRI_num_aver_sounds_ADD_REPEAT = 0

df_aversive_10

,Subject,Sound,Name,Rating,Trigger,Order
0,25,4,Personalized 4,4,No,25
1,25,3,Personalized 3,2,No,44
2,25,6,Personalized 6,2,No,38
3,25,73,m_SippingCoffeeCombined_s.wav,2,No,58


In [21]:
#df_final

In [22]:
#How do these warnings work:
#If there are less than 6 aversive sounds, pull from all a_ sounds that are not misophonic or aversive
#Take all neut/pos sounds from a_ sounds and sort them from least positive to most positive (worst --> best)
#Coerce all sounds to be aversive

#if there are 6 or more aversive sounds, select randomly without repeating and hit 10 aver sounds (no more than 1 repeat per sound)

if warning_MRI_numRows__aversive <= 5:
    warning_MRI_aver_less10 = True
    warning_MRI_numRepeats_aver = 10 - warning_MRI_numRows__aversive #num aver sounds repeated

    aversive_sounds_Aprefix = df_final[(df_final['Sound'] >= 7) & 
                                   (df_final['Sound'] <= 25) & 
                                   (df_final['Trigger'] != 'Yes')].drop('Memory', axis=1) #aversive/pos/neut sounds

    
    #aversiveRatings = aversive_sounds_Aprefix[aversive_sounds_Aprefix['Trigger'] == 'No']
    #aversiveRatings = aversiveRatings.sort_values(by=['Rating'], ascending=True) #.head(warning_MRI_numRepeats_miso)

    posNeutRatings = aversive_sounds_Aprefix[aversive_sounds_Aprefix['Trigger'].isna()]

    posNeutRatings['Rating'] = posNeutRatings['Rating'].astype(int)
    posNeutRatings = posNeutRatings.sort_values(by=['Rating'], ascending=True).head(warning_MRI_numRepeats_aver)
    
    #Coerce
    posNeutRatings['Trigger'] = 'No'

    warning_MRI_aver_sounds_added = set(posNeutRatings['Name']) #which aver sounds added
    warning_MRI_num_aver_sounds_ADD_REPEAT = len(warning_MRI_aver_sounds_added) #num aver sound added 
    
    df_aversive_10 = pd.concat([df_aversive_10, posNeutRatings], ignore_index=True)

    
    
    #aversive_sounds_Aprefix = aversive_sounds_Aprefix[['Subject', 'Sound', 'Name', 'Rating', 'Trigger']]
    #aversive_sounds_Aprefix['Rating'] = aversive_sounds_Aprefix['Rating'].astype(int)
    #aversive_sounds_Aprefix['Rating'] = aversive_sounds_Aprefix['Rating'].abs()
    
    #aversive_sounds_Aprefix = pd.concat([aversive_sounds_Aprefix, df_aversive_10, df_aversive_10]).drop_duplicates(keep=False)
    
    #ADD_aversive_sounds_Aprefix_needed = aversive_sounds_Aprefix.sort_values(by=['Rating'], ascending=True).head(warning_MRI_numRepeats_aver)
    #ADD_aversive_sounds_Aprefix_needed['Trigger'] = 'No'
    #ADD_aversive_sounds_Aprefix_needed['Rating'] = 0
    
    #warning_MRI_aver_sounds_added = set(ADD_aversive_sounds_Aprefix_needed['Name']) #which aver sounds added
    #warning_MRI_num_aver_sounds_ADD_REPEAT = len(warning_MRI_aver_sounds_added) #num aver sound added 
    
    #df_aversive_10 = pd.concat([df_aversive_10, ADD_aversive_sounds_Aprefix_needed], ignore_index=True)


if (warning_MRI_numRows__aversive > 5) and (warning_MRI_numRows__aversive < 10):
    warning_MRI_aver_less10 = True
    warning_MRI_numRepeats_aver = 10 - warning_MRI_numRows__aversive

    DUPLICATES_5to10_MRI = df_aversive_10.sample(n=warning_MRI_numRepeats_aver)
    
    warning_MRI_aver_sounds_repeated = set(DUPLICATES_5to10_MRI['Name'])  #which aver sounds repeated
    warning_MRI_num_aver_sounds_ADD_REPEAT = len(warning_MRI_aver_sounds_repeated) #num aver sound added 
    
    df_aversive_10 = pd.concat([df_aversive_10, DUPLICATES_5to10_MRI], ignore_index=True)

df_aversive_10

,Subject,Sound,Name,Rating,Trigger,Order
0,25,4,Personalized 4,4,No,25
1,25,3,Personalized 3,2,No,44
2,25,6,Personalized 6,2,No,38
3,25,73,m_SippingCoffeeCombined_s.wav,2,No,58
4,25,7,a_AlarmClock_s.wav,0,No,46
5,25,23,a_ToddlerCrying_s_rev.wav,0,No,48
6,25,22,a_ScreamWithEcho_s.wav,0,No,49
7,25,19,a_GhostWhispers_s.wav,0,No,69
8,25,18,a_FlyBuzzing_s.wav,0,No,92
9,25,17,a_FireTruckAlarm_s.wav,0,No,107


##### Create Warning File for MRI #####

In [23]:
# Name of the subdirectory to create within the current directory
subdirectory_name = 'SubjectData/subject_' + str(subjN)
csv_paths = []

# Create the subdirectory if it doesn't exist
if not os.path.exists(subdirectory_name):
    os.makedirs(subdirectory_name)

In [24]:
#if warning is true
if warning_MRI_miso_less10 or warning_MRI_aver_less10:
    #create warning text file. Populate with numDuplicates and what are the duplicates?
    csv_file_path_warning_MRI = os.path.join(subdirectory_name, f'subject_{subjN}_MRI_warning.txt')
    
    with open(csv_file_path_warning_MRI, 'w') as file:
        if warning_MRI_miso_less10:
            file.write("WARNING! SUBJECT DID NOT CLASSIFY 10 UNIQUE SOUNDS AS MISOPHONIC." + "\n\n" + 
                       "Total Number of Classified Misophonic Sounds: " +  str(warning_MRI_numRows__miso) + "\n\n" +
                       "Total Number of Missing Misophonic Sounds: " +  str(warning_MRI_numRepeats_miso) + "\n\n" + 
                       "Repeated Misophonic Sounds: " + str(warning_MRI_miso_sounds_repeated) + "\n\n" +
                       "Added Non-Trigger High Arousal 'Misophonic' (m_) Sounds: " + str(warning_MRI_miso_sounds_added) + "\n\n" +
                       "Total Number of Repeated/Added Misophonic Sounds: " + str(warning_MRI_num_miso_sounds_ADD_REPEAT) + "\n\n\n\n\n")
            
        if warning_MRI_aver_less10:
            file.write("WARNING! SUBJECT DID NOT CLASSIFY 10 UNIQUE SOUNDS AS AVERSIVE." + "\n\n" + 
                       "Total Number of Classified Misophonic Sounds: " +  str(warning_MRI_numRows__aversive) + "\n\n" +
                       "Total Number of Missing Aversive Sounds: " +  str(warning_MRI_numRepeats_aver) + "\n\n" + 
                       "Repeated Aversive Sounds: " + str(warning_MRI_aver_sounds_repeated) + "\n\n" +
                       "Added Positive/Neutral Low Arousal 'Aversive' (a_) Sounds: " + str(warning_MRI_aver_sounds_added) + "\n\n" +
                       "Total Number of Repeated/Added Misophonic Sounds: " + str(warning_MRI_num_aver_sounds_ADD_REPEAT) + "\n\n")
            
        
    print(f"File saved to {csv_file_path_warning_MRI}")
    csv_paths.append(csv_file_path_warning_MRI)

File saved to SubjectData/subject_25/subject_25_MRI_warning.txt


In [25]:
df_mri_ratings = pd.concat([df_miso_10, df_aversive_10], ignore_index=True)
df_mri_ratings

,Subject,Sound,Name,Rating,Trigger,Order
0,25,75,m_SlurpingEatingCombined_s.wav,7,Yes,80
1,25,65,m_IADS_Vomit_255_s.wav,6,Yes,34
2,25,49,m_EatingSmallCrunches_s.wav,4,Yes,11
3,25,59,m_HotTeaSlurping_s.wav,4,Yes,21
4,25,76,m_SlurpingNoodles_s.wav,4,Yes,109
5,25,73,m_SippingCoffeeCombined_s.wav,2,Yes,58
6,25,26,m_ALotOfYawning_s.wav,0,Yes,67
7,25,56,m_HeavyBreathing2_s.wav,0,Yes,81
8,25,57,m_HeavyBreathing_s.wav,0,Yes,95
9,25,60,m_iads_Chewing_724_s.wav,0,Yes,66


#### 5.1. Create Rankings/Order_labels For Nimesha ####

In [26]:
#Create Ranking

# Group by 'Trigger', rank each group, and map the rankings back to the original DataFrame.
df_mri_ratings['Rating'] = df_mri_ratings['Rating'].astype(int)
df_mri_ratings['Ranking'] = df_mri_ratings.groupby('Trigger')['Rating'].rank(ascending=True, method='first').astype(int)

# Now sort by 'Trigger' and then by 'Ranking' within each group to see the results.
df_mri_ratings.sort_values(by=['Trigger', 'Ranking'], inplace=True)

df_mri_ratings = df_mri_ratings.reset_index(drop=True)
#df_mri_ratings

In [27]:
#Create Order Label
dictionary1 = {1: "A",
              2: "B",
              3: "C",
              4: "D",
              5: "E",
              6: "F",
              7: "G",
              8: "H",
              9: "I",
              10: "J" }

dictionary2 = {"A": 1,
               "B": 5,
               "C": 2,
               "D": 6,
               "E": 3,
               "F": 7,
               "G": 9,
               "H": 4,
               "I": 8,
               "J": 10}

dictionary11 = {1: "A",
              2: "C",
              3: "E",
              4: "H",
              5: "B",
              6: "D",
              7: "F",
              8: "I",
              9: "G",
              10: "J" }

dictionary22 = {"A": 1,
               "B": 2,
               "C": 3,
               "D": 4,
               "E": 5,
               "F": 6,
               "G": 7,
               "H": 8,
               "I": 9,
               "J": 10}

df_mri_ratings['Alpha'] = df_mri_ratings['Ranking'].map(dictionary11)
df_mri_ratings['Order_Label'] = df_mri_ratings['Alpha'].map(dictionary22)

#df_mri_ratings

#### 5.1. Create df_mri_ratings_nimesha: Ratings CSV file to be sent to nimesha ####

In [28]:
column_to_remove = ['Subject', 'Sound', 'Alpha', 'Order']
df_mri_ratings_nimesha = df_mri_ratings.drop(columns=column_to_remove)

df_mri_ratings_nimesha

,Name,Rating,Trigger,Ranking,Order_Label
0,a_AlarmClock_s.wav,0,No,1,1
1,a_ToddlerCrying_s_rev.wav,0,No,2,3
2,a_ScreamWithEcho_s.wav,0,No,3,5
3,a_GhostWhispers_s.wav,0,No,4,8
4,a_FlyBuzzing_s.wav,0,No,5,2
5,a_FireTruckAlarm_s.wav,0,No,6,4
6,Personalized 3,2,No,7,6
7,Personalized 6,2,No,8,9
8,m_SippingCoffeeCombined_s.wav,2,No,9,7
9,Personalized 4,4,No,10,10


#### 5.2 Create df_mri_sound_names: Just 20 MRI Sound Names #### 

In [29]:
df_mri_sound_names = df_mri_ratings['Name']
df_mri_sound_names

0                 a_AlarmClock_s.wav
1          a_ToddlerCrying_s_rev.wav
2             a_ScreamWithEcho_s.wav
3              a_GhostWhispers_s.wav
4                 a_FlyBuzzing_s.wav
5             a_FireTruckAlarm_s.wav
6                     Personalized 3
7                     Personalized 6
8      m_SippingCoffeeCombined_s.wav
9                     Personalized 4
10             m_ALotOfYawning_s.wav
11           m_HeavyBreathing2_s.wav
12            m_HeavyBreathing_s.wav
13          m_iads_Chewing_724_s.wav
14     m_SippingCoffeeCombined_s.wav
15       m_EatingSmallCrunches_s.wav
16            m_HotTeaSlurping_s.wav
17           m_SlurpingNoodles_s.wav
18            m_IADS_Vomit_255_s.wav
19    m_SlurpingEatingCombined_s.wav
Name: Name, dtype: object

## 6. Create df_tms_ratings: 24 tms sounds ratings table ##

#### Warning: If there are <12 miso sounds and <24 miso sounds... ####

In [30]:
warning_TMS_less12 = False
warning_TMS_less24 = False

numMiso = df_miso.shape[0]

num_Mprefix_needed = 0

df_miso

,Subject,Sound,Name,Rating,Trigger,Order
0,25,49,m_EatingSmallCrunches_s.wav,4,Yes,11
1,25,59,m_HotTeaSlurping_s.wav,4,Yes,21
2,25,65,m_IADS_Vomit_255_s.wav,6,Yes,34
3,25,75,m_SlurpingEatingCombined_s.wav,7,Yes,80
4,25,76,m_SlurpingNoodles_s.wav,4,Yes,109


In [31]:
#How do these warnings work:
#If there are less than 12 misophonic sounds, pull from all m_ sounds.
#Take all aversive sounds from m_ sounds and sort them from most negative to least negative (most negative --> least negative)
#Take all neut/pos sounds from m_ sounds and sort them from least positive to most positive (most negative --> least negative)
#Concat aversive --> neut/pos sounds together and take only the most negative x sounds needed to hit 12 sounds
#Coerce all sounds to be miso and change all neg values to abs values
#Duplicate these 12 sounds (repeating each sound once) to hit 24

if numMiso < 12: #if we need to add miso non-trigger sounds
    num_Mprefix_needed = 12 - numMiso
    #Create Highest Aversive Rating of Sounds 26-98 "m_...wav"

    nontriggerMiso_sounds_Mprefix = df_final[(df_final['Sound'] >= 26) & (df_final['Sound'] <= 95)].drop('Memory', axis=1)

    aversiveRatings = nontriggerMiso_sounds_Mprefix[nontriggerMiso_sounds_Mprefix['Trigger'] == 'No']
    aversiveRatings = aversiveRatings.sort_values(by=['Rating'], ascending=True) #.head(warning_MRI_numRepeats_miso)

    posNeutRatings = nontriggerMiso_sounds_Mprefix[nontriggerMiso_sounds_Mprefix['Trigger'].isna()]

    posNeutRatings['Rating'] = posNeutRatings['Rating'].astype(int)
    posNeutRatings = posNeutRatings.sort_values(by=['Rating'], ascending=True) #.head(warning_MRI_numRepeats_miso)
    #print(posNeutRatings)
    
    aversive_posNeut_Ratings = pd.concat([aversiveRatings, posNeutRatings])
    #print(aversive_posNeut_Ratings)
    aversive_posNeut_Ratings = aversive_posNeut_Ratings.head(num_Mprefix_needed)
    #print(aversive_posNeut_Ratings)

    #Coerce
    aversive_posNeut_Ratings['Trigger'] = 'Yes'
    aversive_posNeut_Ratings['Rating'] = aversive_posNeut_Ratings['Rating'].astype(int)
    aversive_posNeut_Ratings['Rating'] = aversive_posNeut_Ratings['Rating'].abs()
    #print(aversive_posNeut_Ratings)

    warning_Miso_added = set(aversive_posNeut_Ratings['Name'])

    df_tms_ratings = pd.concat([df_miso, aversive_posNeut_Ratings], ignore_index=True)

    warning_Miso_repeated = set(df_miso['Name'])
    
    df_tms_ratings_copy = df_tms_ratings.copy()
    df_tms_ratings = pd.concat([df_tms_ratings, df_tms_ratings_copy], ignore_index=True)
    
    #coerce every trigger to be yes
    df_tms_ratings['Trigger'] = 'Yes'
    df_tms_ratings['Rating'] = df_tms_ratings['Rating'].astype(int)
    df_tms_ratings = df_tms_ratings.sort_values(by=['Rating'], ascending=True)

    category = [0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3]
    df_tms_ratings['Category'] = category


    warning_TMS_less12 = True

    
    # #aversive_sounds_Mprefix = df_aversive[(df_aversive['Sound'] >= 26) & (df_aversive['Sound'] <= 95)]
    # #aversive_sounds_Mprefix = df_final[(df_final['Sound'] >= 26) & (df_final['Sound'] <= 95)].drop('Memory', axis=1)

    # nontriggerMiso_sounds_Mprefix = df_final[(df_final['Sound'] >= 26) & (df_final['Sound'] <= 95)].drop('Memory', axis=1)

    # nontriggerMiso_sounds_Mprefix['Rating'] = nontriggerMiso_sounds_Mprefix['Rating'].astype(int)
    # nontriggerMiso_sounds_Mprefix['Rating'] = nontriggerMiso_sounds_Mprefix['Rating'].abs()
    
    # aversive_sounds_Mprefix = pd.concat([nontriggerMiso_sounds_Mprefix, df_miso_10, df_miso_10]).drop_duplicates(keep=False)
    
    # DUPLICATES_aversive_sounds_Mprefix_needed = aversive_sounds_Mprefix.sort_values(by=['Rating'], ascending=True).head(num_Mprefix_needed)
    # DUPLICATES_aversive_sounds_Mprefix_needed['Trigger'] = 'Yes'
    # DUPLICATES_aversive_sounds_Mprefix_needed['Rating'] = 0

    # #DUPLICATES_aversive_sounds_Mprefix_needed = aversive_sounds_Mprefix.sort_values(by=['Rating'], ascending=False).head(num_Mprefix_needed)
    # #DUPLICATES_aversive_sounds_Mprefix_needed = aversive_sounds_Mprefix.sort_values(by=['Rating'], ascending=True).head(num_Mprefix_needed)
    
    # warning_Miso_added = set(DUPLICATES_aversive_sounds_Mprefix_needed['Name'])

    # df_tms_ratings = pd.concat([df_miso, DUPLICATES_aversive_sounds_Mprefix_needed], ignore_index=True)

    # warning_Miso_repeated = set(df_miso['Name'])
    
    # df_tms_ratings_copy = df_tms_ratings.copy()
    # df_tms_ratings = pd.concat([df_tms_ratings, df_tms_ratings_copy], ignore_index=True)
    
    # #coerce every trigger to be yes
    # df_tms_ratings['Trigger'] = 'Yes'
    # df_tms_ratings['Rating'] = df_tms_ratings['Rating'].astype(int)
    # df_tms_ratings = df_tms_ratings.sort_values(by=['Rating'], ascending=True)

    # category = [0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3]
    # df_tms_ratings['Category'] = category


    # warning_TMS_less12 = True

/var/folders/qq/qm2zshms6rl57mb9s95rh3xc0000gn/T/ipykernel_4141/290087168.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posNeutRatings['Rating'] = posNeutRatings['Rating'].astype(int)


In [32]:
#if there are between 13 and 23, select randomly without repeating and hit 24 miso sounds (no more than 1 repeat per sound)
if (numMiso >= 12) and (numMiso < 24):
    num_Mprefix_needed = 24 - numMiso
    DUPLICATES_12to24_tms = df_miso.sample(n=num_Mprefix_needed)

    warning_Miso_repeated = set(DUPLICATES_12to24_tms['Name'])
    
    df_tms_ratings = pd.concat([df_miso, DUPLICATES_12to24_tms], ignore_index=True)

    #coerce every trigger to be yes
    df_tms_ratings['Trigger'] = 'Yes'
    df_tms_ratings['Rating'] = df_tms_ratings['Rating'].astype(int)
    df_tms_ratings = df_tms_ratings.sort_values(by=['Rating'], ascending=True)

    category = [0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3]
    df_tms_ratings['Category'] = category

    warning_TMS_less24 = True

In [33]:
if numMiso >= 24:
    #create 6 personalized, and then choose 6 top and 6 bottom from nonpersonalized
    #change this
    df_miso_personalized = df_final[df_final['Sound'] <= 6]
    #df_miso_personalized = df_miso[df_miso['Sound'] <= 6]
    
    df_miso_noPersonalized = df_miso[df_miso['Sound'] > 6]
    df_miso_NP_6Highest = df_miso_noPersonalized.sort_values(by=['Rating'], ascending = False).head(6).reset_index(drop=True)
    df_miso_NP_6Lowest = df_miso_noPersonalized.sort_values(by=['Rating'], ascending = True).head(6).reset_index(drop=True)
    
    #Get rid of top 6 and bottom 6 sounds to pull random sounds for middle 6
    NP_Or6Highest = pd.concat([df_miso_noPersonalized, df_miso_NP_6Highest, df_miso_NP_6Highest]).drop_duplicates(keep=False)
    NP_Or6Highest_Or6Lowest = pd.concat([NP_Or6Highest, df_miso_NP_6Lowest, df_miso_NP_6Lowest]).drop_duplicates(keep=False)
    df_miso_NP_6Middle = NP_Or6Highest_Or6Lowest.sample(n=6)
    
    #concat all together
    df_tms_ratings = pd.concat([df_miso_NP_6Lowest, df_miso_NP_6Middle, df_miso_NP_6Highest, df_miso_personalized], ignore_index=True)
    category = [0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3]
    df_tms_ratings['Category'] = category

##### Generate Warning Files ######

In [34]:
#if warning is true
if warning_TMS_less12 or warning_TMS_less24:
    #create warning text file. Populate with numDuplicates and what are the duplicates?
    csv_file_path_warning_TMS = os.path.join(subdirectory_name, f'subject_{subjN}_TMS_warning.txt')
    
    with open(csv_file_path_warning_TMS, 'w') as file:
        if warning_TMS_less12:
            file.write("WARNING! SUBJECT CLASSIFIED LESS THAN 12 UNIQUE SOUNDS AS MISOPHONIC." + "\n\n" + 
                       "Total Number of Classified Misophonic Sounds: " +  str(numMiso) + "\n\n" +
                       "Total Number of Missing Misophonic Sounds: " +  str(num_Mprefix_needed+12) + "\n\n" + 
                       "Added Non-Trigger High Arousal 'Misophonic' (m_) Sounds: " + str(warning_Miso_added) + "\n\n" +
                       "Repeated Misophonic Sounds (Repeated Once): " + str(warning_Miso_repeated) + "\n\n" +
                       "Total Number of Repeated/Added Misophonic Sounds: " + str(num_Mprefix_needed+12) + "\n\n\n\n\n")
        else: #12 to 24 sounds
            file.write("WARNING! SUBJECT CLASSIFIED LESS THAN 24 UNIQUE SOUNDS AS MISOPHONIC." + "\n\n" + 
                       "Total Number of Classified Misophonic Sounds: " +  str(numMiso) + "\n\n" +
                       "Total Number of Missing Misophonic Sounds: " +  str(num_Mprefix_needed) + "\n\n" + 
                       "Repeated Misophonic Sounds: " + str(warning_Miso_repeated) + "\n\n" +
                       "Added Non-Trigger High Arousal 'Misophonic' (m_) Sounds: " + "None" + "\n\n" +
                       "Total Number of Repeated/Added Misophonic Sounds: " + str(num_Mprefix_needed) + "\n\n\n\n\n")
            
        
    print(f"File saved to {csv_file_path_warning_TMS}")
    csv_paths.append(csv_file_path_warning_TMS)

File saved to SubjectData/subject_25/subject_25_TMS_warning.txt


In [35]:
df_tms_ratings_FINAL = df_tms_ratings[['Subject', 'Category', 'Name', 'Trigger', 'Rating']]
df_tms_ratings_FINAL

,Subject,Category,Name,Trigger,Rating
11,25,0,m_iads_Hiccup_245_s.wav,Yes,0
21,25,0,m_iads_Chewing_724_s.wav,Yes,0
20,25,0,m_HeavyBreathing_s.wav,Yes,0
19,25,0,m_HeavyBreathing2_s.wav,Yes,0
18,25,0,m_ALotOfYawning_s.wav,Yes,0
22,25,0,m_IADS_FemaleCough_242_s.wav,Yes,0
10,25,1,m_IADS_FemaleCough_242_s.wav,Yes,0
9,25,1,m_iads_Chewing_724_s.wav,Yes,0
23,25,1,m_iads_Hiccup_245_s.wav,Yes,0
7,25,1,m_HeavyBreathing2_s.wav,Yes,0


#### 6.2 Create df_tms_sound_names: 24 TMS Sound Names #### 

In [36]:
df_tms_sound_names = df_tms_ratings['Name']
df_tms_sound_names

11           m_iads_Hiccup_245_s.wav
21          m_iads_Chewing_724_s.wav
20            m_HeavyBreathing_s.wav
19           m_HeavyBreathing2_s.wav
18             m_ALotOfYawning_s.wav
22      m_IADS_FemaleCough_242_s.wav
10      m_IADS_FemaleCough_242_s.wav
9           m_iads_Chewing_724_s.wav
23           m_iads_Hiccup_245_s.wav
7            m_HeavyBreathing2_s.wav
6              m_ALotOfYawning_s.wav
8             m_HeavyBreathing_s.wav
5      m_SippingCoffeeCombined_s.wav
17     m_SippingCoffeeCombined_s.wav
12       m_EatingSmallCrunches_s.wav
13            m_HotTeaSlurping_s.wav
16           m_SlurpingNoodles_s.wav
4            m_SlurpingNoodles_s.wav
1             m_HotTeaSlurping_s.wav
0        m_EatingSmallCrunches_s.wav
14            m_IADS_Vomit_255_s.wav
2             m_IADS_Vomit_255_s.wav
15    m_SlurpingEatingCombined_s.wav
3     m_SlurpingEatingCombined_s.wav
Name: Name, dtype: object

## ------------------------------------------------------------------------------ ##

## Save Miso and Non-Miso DFs as CSVs to SubjectData/SubjectNum ##

### Deal with Warnings ###

In [37]:
# Define file path
csv_file_path_20MRI_sounds = os.path.join(subdirectory_name, f'subject_{subjN}_20MRI_sounds_names.csv')
csv_file_path_sound_rating_MRI = os.path.join(subdirectory_name, f'subject_{subjN}_20MRI_sounds_ratings.csv')
csv_file_path_24TMS_sounds = os.path.join(subdirectory_name, f'subject_{subjN}_24TMS_sounds_names.csv')
csv_file_path_sound_rating_TMS = os.path.join(subdirectory_name, f'subject_{subjN}_24TMS_sounds_ratings.csv')
csv_file_path_sound_rating_ALL = os.path.join(subdirectory_name, f'subject_{subjN}_ALL_sounds_ratings.csv')


# Save the Sound column to CSV
df_mri_sound_names.to_csv(csv_file_path_20MRI_sounds, index=False, header=False)
df_mri_ratings_nimesha.to_csv(csv_file_path_sound_rating_MRI, index=False, header=True)
df_tms_sound_names.to_csv(csv_file_path_24TMS_sounds, index=False, header=False)
df_tms_ratings_FINAL.to_csv(csv_file_path_sound_rating_TMS, index=False, header=True)
sound_rating_all.to_csv(csv_file_path_sound_rating_ALL, index=False, header=True)

# Collecting file paths for download links
csv_paths.append(csv_file_path_20MRI_sounds)
csv_paths.append(csv_file_path_sound_rating_MRI)
csv_paths.append(csv_file_path_24TMS_sounds)
csv_paths.append(csv_file_path_sound_rating_TMS)
csv_paths.append(csv_file_path_sound_rating_ALL)
    

csv_paths

['SubjectData/subject_25/subject_25_MRI_warning.txt',
 'SubjectData/subject_25/subject_25_TMS_warning.txt',
 'SubjectData/subject_25/subject_25_20MRI_sounds_names.csv',
 'SubjectData/subject_25/subject_25_20MRI_sounds_ratings.csv',
 'SubjectData/subject_25/subject_25_24TMS_sounds_names.csv',
 'SubjectData/subject_25/subject_25_24TMS_sounds_ratings.csv',
 'SubjectData/subject_25/subject_25_ALL_sounds_ratings.csv']